# Coding Discussion 4

In [1]:
#load in packages
import numpy as np
import pandas as pd

In [2]:
with open('../Data/aljazeera-khashoggi.txt') as f:
    alj = f.read()

with open('../Data/BBC-khashoggi.txt') as f:
    bbc = f.read()
    
with open('../Data/breitbart-khashoggi.txt', encoding='utf8') as f:
    bre = f.read()
    
with open('../Data/fox-khashoggi.txt', encoding='utf8') as f:
    fox = f.read()
    
with open('../Data/cnn-khashoggi.txt') as f:
    cnn = f.read()

In [3]:
def tokenize(text=None):
    """
    tokenize converts text into a list of words while removing non-letter characters.
    It takes a string and returns a list of strings.
    """
    text = text.lower()
    text = text.replace('.',' ')
    text = text.replace(',', ' ')
    text = text.replace('(', ' ')
    text = text.replace(')', ' ')
    text = text.replace("’", ' ')
    text = text.replace("'", ' ')
    text = text.replace(':', ' ')
    text = text.replace('“', ' ')
    text = text.replace('”', ' ')
    text = text.replace('—', ' ')
    text = text.replace('"', ' ')
    text = text.replace('[', ' ')
    text = text.replace(']', ' ')
    text = text.replace('?', ' ')
    text = text.replace('!',' ')
    text = text.replace('/',' ')
    text = text.replace('\\', ' ')
    text = text.replace('-', ' ')
    text = text.replace('$', '')
    text = text.replace('£', '')
    text = text.replace('0', '')
    text = text.replace('1', '')
    text = text.replace('2', '')
    text = text.replace('3', '')
    text = text.replace('4', '')
    text = text.replace('5', '')
    text = text.replace('6', '')
    text = text.replace('7', '')
    text = text.replace('8', '')
    text = text.replace('9', '')
    text_list = text.split()
    return text_list

In [4]:
def convert_text_to_dtm(txt):
    '''
    Converts text into a document term matrix.
    '''
    d = dict()
    for word in tokenize(txt):
        if word in d:
            d[word][0] += 1
        else:
            d[word] = [1]
    return pd.DataFrame(d)

In [5]:
def gen_DTM(texts=None):
    '''
    Generate a document term matrix
    '''
    DTM = pd.DataFrame()
    for text in texts:
        entry = convert_text_to_dtm(text)
        DTM = DTM.append(pd.DataFrame(entry),ignore_index=True,sort=True) # Row bind
    
    DTM.fillna(0, inplace=True) # Fill in any missing values with 0s (i.e. when a word is in one text but not another)
    return DTM

In [6]:
df = gen_DTM([alj, bbc, bre, fox, cnn])
df

,a,abdulaziz,able,about,absent,accident,accidentally,according,account,accounts,...,world,worse,would,writer,yalova,year,yelova,yet,your,â
0,11,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23,0.0,0.0,2,1.0,0.0,0.0,1.0,1.0,2.0,...,1.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,3.0,1.0
2,11,2.0,0.0,2,0.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,16,0.0,0.0,4,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0
4,14,0.0,1.0,1,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [7]:
def cosine_fun(x,y):
    """
    cosine_fun takes two arrays and finds the cosine 
    """
    cos= (np.dot(x,y)/( np.sqrt(np.dot(x,x)) * np.sqrt(np.dot(y,y)) ))
    return cos

In [8]:
#creating a dictionary for each article as an array of its values
ndict = {'alj':df.iloc[0].values,
'bbc':df.iloc[1].values,
'bre':df.iloc[2].values,
'cnn':df.iloc[3].values,
'fox':df.iloc[4].values}
#turning dictionary into a new data frame 
ndf = pd.DataFrame(ndict)
#finding the cosines for each pair of articles using the correlation command
cosines = ndf.corr(method=cosine_fun)
cosines

,alj,bbc,bre,cnn,fox
alj,1.000000,0.874243,0.841171,0.847628,0.765426
bbc,0.874243,1.000000,0.900848,0.891444,0.759504
bre,0.841171,0.900848,1.000000,0.869250,0.703809
cnn,0.847628,0.891444,0.869250,1.000000,0.768582
fox,0.765426,0.759504,0.703809,0.768582,1.000000


With stop words included but non-letter characters removed, these articles seem fairly similar. Fox has the fewest similiarities to other articles. Brietbart and the BBC talk abuot the scandal in the most similar ways, while Breitbart and Fox are most dissimilar. 

In [9]:
#reading in stop words and formatting them as a list
stopwords = pd.read_csv("../Data/stop_words.csv")
stopwords = stopwords['word'].tolist()


#rewriting tokenize function to exclude stop words 
def tokenize_stop(text=None):
    """
    tokenize converts text into a list of words while removing non-letter characters.
    It takes a string and returns a list of strings.
    """
    text = text.lower()
    text = text.replace('.',' ')
    text = text.replace(',', ' ')
    text = text.replace('(', ' ')
    text = text.replace(')', ' ')
    text = text.replace("’", ' ')
    text = text.replace("'", ' ')
    text = text.replace(':', ' ')
    text = text.replace('“', ' ')
    text = text.replace('”', ' ')
    text = text.replace('—', ' ')
    text = text.replace('"', ' ')
    text = text.replace('[', ' ')
    text = text.replace(']', ' ')
    text = text.replace('?', ' ')
    text = text.replace('!',' ')
    text = text.replace('/',' ')
    text = text.replace('\\', ' ')
    text = text.replace('-', ' ')
    text = text.replace('$', '')
    text = text.replace('£', '')
    text = text.replace('0', '')
    text = text.replace('1', '')
    text = text.replace('2', '')
    text = text.replace('3', '')
    text = text.replace('4', '')
    text = text.replace('5', '')
    text = text.replace('6', '')
    text = text.replace('7', '')
    text = text.replace('8', '')
    text = text.replace('9', '')
    text_list = text.split()
    text_list2 = [word for word in text_list if word not in stopwords]
    return text_list2

#new convert text to dtm function to exclude stop words
def convert_text_to_dtm_stop(txt):
    '''
    Converts text into a document term matrix.
    '''
    d = dict()
    for word in tokenize_stop(txt):
        if word in d:
            d[word][0] += 1
        else:
            d[word] = [1]
    return pd.DataFrame(d)


#new document term matrix function to exlude stop words
def gen_DTM_stop(texts=None):
    '''
    Generate a document term matrix
    '''
    DTM = pd.DataFrame()
    for text in texts:
        entry = convert_text_to_dtm_stop(text)
        DTM = DTM.append(pd.DataFrame(entry),ignore_index=True,sort=True) # Row bind
    
    DTM.fillna(0, inplace=True) # Fill in any missing values with 0s (i.e. when a word is in one text but not another)
    return DTM

In [10]:
df2 = gen_DTM_stop([alj, bbc, bre, fox, cnn])
df2

,abdulaziz,absent,accident,accidentally,account,accounts,accusation,accusing,acknowledged,added,...,white,widely,withheld,woods,world,worse,writer,yalova,yelova,â
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,2.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,0.0
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
#creating a dictionary for each article as an array of its values
ndict2 = {'alj':df2.iloc[0].values,
'bbc':df2.iloc[1].values,
'bre':df2.iloc[2].values,
'cnn':df2.iloc[3].values,
'fox':df2.iloc[4].values}
#turning dictionary into a new data frame 
ndf2 = pd.DataFrame(ndict2)
#finding the cosines for each pair of articles using the correlation command
unstoppedcosines = ndf2.corr(method=cosine_fun)
unstoppedcosines

,alj,bbc,bre,cnn,fox
alj,1.000000,0.704096,0.601929,0.721042,0.610199
bbc,0.704096,1.000000,0.622375,0.694799,0.551647
bre,0.601929,0.622375,1.000000,0.582348,0.424620
cnn,0.721042,0.694799,0.582348,1.000000,0.608453
fox,0.610199,0.551647,0.424620,0.608453,1.000000


Without stop words included, the articles are overall less similar to each other. Fox and Breitbart are still the most dissimilar, but Breitbart andd the BBC are no longer very similar. Instead, CNN and Aljazeera are the most similar among articles. 

Although removing stop words has decreased the similarity between these articles, using sentiment analysis would be helpful to pinpoint how similar or dissimilar they truly are. Additionally, removing words that are essential to the story but do not carry emotional information (such as names) may help in further exploring similarities.